pandas 1.5.2 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.19.5 which is incompatible.
ImportError: this version of pandas is incompatible with numpy < 1.20.3
your numpy version is 1.19.5.
Please upgrade numpy to >= 1.20.3 to use this pandas version

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simcse 0.4 requires numpy<1.20,>=1.19.5, but you have numpy 1.20.3 which is incompatible.


conda search --info pandas | grep -E '(^version|numpy)'
version     : 1.3.4
  - numpy >=1.19.5,<2.0a0

en un mot, downgrade pandas to 1.3.4

In [ ]:
# !pip install --upgrade numpy==1.19.5
# !pip install --upgrade pandas==1.3.4

In [ ]:
import re
import numpy as np
import pandas as pd
import json
import math
import random
import copy
import json

from simcse import SimCSE
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = SimCSE("princeton-nlp/sup-simcse-roberta-large")

In [ ]:
# #### load and prepocess raw data
# def make_sentence(row):
#     return ' '.join([str(row['head_name']),str(row['relation']),str(row['tail_name'])])
# def trim_relations(rel):
#     return ', '.join(rel.split('/')[1:])
    
# df = pd.read_json('./train_df.json')
# df['relation'] = df['relation'].apply(trim_relations)
# df['sentence'] = df.apply(make_sentence, axis=1)
# # df['tokens'] = df.apply(lambda hrt: tokenizer.tokenize(hrt['sentence']),axis = 1) # padas dataframe doesn't support list datatype.
# tokenize_sentences =  {str(index): tokenizer.tokenize(sentence) for index, sentence in df['sentence'].items()}

# #### save data
# df.to_csv('./train_df_token.csv',index=False) 
# with open('tokenize_sentences.json', 'w') as fp:
#     json.dump(tokenize_sentences, fp)

In [ ]:
### load data
df = pd.read_csv('./train_df_token.csv')
with open('tokenize_sentences.json', 'r') as fp:
    tokenize_sentences = json.load(fp)

In [ ]:
tokens_len = pd.Series({index: len(tokens) for index, tokens in tokenize_sentences.items()})
dict_merged_tokens = {index: str(hrt) for index, hrt in tokenize_sentences.items()}

已知一個hrt，假設tail被mask。
找他的one hop

In [ ]:
i = 0
hrt = df.iloc[i] # a given hrt. Assume tail is masked

#### find 1-hops
one_hops = df[(df.tail_code == hrt['tail_code']) & (df.index != i)] # itself is not its 1-hop

token數量不能大於512，
所以如果大於512，讓one hops 和hrt比較similarity，從高similarity的優先取，取到超過(一超過就不取了，沒有再去看低similarity的onehop有沒有符合)

In [ ]:
token_size =  tokens_len[i] + tokens_len[one_hops.index].sum() + len(one_hops.index)
# token_size =  hrt['tokens_len'] + one_hops['tokens_len'].sum() + len(one_hops.index)
if token_size > 512:    # check if token size is greater than 512, choose those one hops with high similarity
    one_hops['similarity'] = one_hops.apply(lambda row: model.similarity(hrt['sentence'], row['sentence']),axis=1)
    one_hops = one_hops.sort_values(by='similarity',ascending=False)
    accum =  tokens_len[one_hops.index].cumsum()
    sep = pd.Series(list(range(1, accum.size+1)), index=accum.index) # use n [SEP] tokens to seperate n one-hop
    accum = accum.add(sep).add(tokens_len[i])
    
one_hops=one_hops[(accum <= 512).values]        # filter out
token_size_after =  tokens_len[i] + tokens_len[one_hops.index].sum() + len(one_hops.index) # check again
# if token_size_after >= 512:
#     print('error')


hrt間用[SEP]區隔最後輸出

In [ ]:
# hrts = hrt['tokens'] + [token for token_list in one_hops['tokens'].tolist() for token in token_list]
## Given a list of lists l, flat_list = [item for sublist in l for item in sublist]
# hrts = tokenize_sentences[1] + [tokens  for index in one_hops.index for tokens in tokenize_sentences[index]]
# dict_merged_tokens = {index: '[D]'.join(hrt) for index, hrt in tokenize_sentences.items()}

hrts_token =  [dict_merged_tokens[str(i)]] + [dict_merged_tokens[str(index)]  for index in one_hops.index]
output = '[SEP] \n'.join(hrts_token) # seperate hrts with [SEP]

In [ ]:
print(output)